In [1]:
code = {
    "name": "commands",
    "body": [
        {
            "name": "repeat-n",
            "count": 10,
            "body": [
                {
                    "name": "gpio-write",
                    "pin": 16,
                    "value": 1
                },
                {
                    "name": "wait-sec",
                    "duration": 1
                },
                {
                    "name": "gpio-write",
                    "pin": 16,
                    "value": 0
                },
                {
                    "name": "wait-sec",
                    "duration": 1
                }
            ]
        }
    ]
}

In [2]:
from proc_comp import parser

p = parser.parse(code)

#print(p)
print(p.pprint())

SeqExp([
  RepeatExp((U32) 10, [
    SeqExp([
      ProcSetExp(gpio_mode[16], (U8) 1)
      ProcSetExp(gpio_value[16], (U8) 1)
    ])
    WaitTimeExp((U32) 1)
    SeqExp([
      ProcSetExp(gpio_mode[16], (U8) 1)
      ProcSetExp(gpio_value[16], (U8) 0)
    ])
    WaitTimeExp((U32) 1)
  ])
])


In [3]:
from proc_comp.codegen.codegen import CodeGen

G = CodeGen()
G.code_gen(p)


Procedures:
proc$1
	ProcSet gpio_mode[16] (U8) 1
	ProcSet gpio_value[16] (U8) 1
	ProcUnop time idt (UInt32) param$U32$1
	ProcBinop (UInt32) param$U32$1 + (U32) 1 (UInt32) param$U32$1
	ProcBlock time >= (UInt32) param$U32$1
	ProcSet gpio_mode[16] (U8) 1
	ProcSet gpio_value[16] (U8) 0
	ProcUnop time idt (UInt32) param$U32$2
	ProcBinop (UInt32) param$U32$2 + (U32) 1 (UInt32) param$U32$2
	ProcBlock time >= (UInt32) param$U32$2
proc$2
	ProcSet (UInt32) param$U32$3 (U32) 0
	ProcSet (UInt32) param$U32$4 (U32) 10
	ProcCall proc$1
	ProcUnop (UInt32) param$U32$3 ++ (UInt32) param$U32$3
	ProcIfElse (UInt32) param$U32$3 < (UInt32) param$U32$4
	ProcCall proc$2
	ProcNoop
Main:
	ProcCall proc$2


In [4]:
G.allocate_registers()

AttributeError: 'CodeGen' object has no attribute 'allocate_registers'

In [ ]:
import types

class Ty:
    __match_args__ = ("a", "b")
    def __init__(self, a, b):
        self.a = a
        self.b = b

t = Ty(1, 1)

match t:
    case Ty(isinstance(a, types.int), isinstance(b, types.int)):
        print("ints")
    case Ty(isinstance(a, types.int), _):
        print("Only a is int")
    case Ty(_, isinstance(b, types.int)):
        print("Only b is int")
    case _:
        print("neither int")

TypeError: called match pattern must be a type

json -parser-> AbSyn -codegen-> ISH 